In [1]:

import json
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def jsonl_to_df(jsonl_file):
    texts = []
    slot_seqs = []
    intents = []

    with open(jsonl_file, "r") as f:
        for line in f:
            data = json.loads(line)
            sentence = data["sentence"]
            tokens = sentence.split()
            slots = ["O"] * len(tokens)

            for entity in data.get("entities", []):
                span = entity["span"]
                typ = entity["type"]
                for i, token_idx in enumerate(span):
                    if token_idx < len(tokens):
                        if i == 0:
                            slots[token_idx] = f"B-{typ}"
                        else:
                            slots[token_idx] = f"I-{typ}"

            texts.append(sentence)
            slot_seqs.append(" ".join(slots))
            intents.append(data["intent"])

    df = pd.DataFrame({"text": texts, "slots": slot_seqs, "intent": intents})
    return df

train_df = jsonl_to_df("train.jsonl")
val_df = jsonl_to_df("devel.jsonl")
test_df = jsonl_to_df("test.jsonl")


all_tokens = []
for sentence in train_df['text']:
    all_tokens.extend(sentence.split())
vocab = sorted(set(all_tokens))
word2idx = {"<pad>":0, "<unk>":1}
for i, w in enumerate(vocab, start=2):
    word2idx[w] = i

all_slots = set()
for slots in train_df['slots']:
    all_slots.update(slots.split())
slot2id = {"O":0}
i = 1
for s in all_slots:
    if s != "O":
        slot2id[s] = i
        i +=1

all_intents = train_df['intent'].unique()
intent2id = {intent:i for i,intent in enumerate(all_intents)}

id2slot = {v:k for k,v in slot2id.items()}
id2intent = {v:k for k,v in intent2id.items()}


class SLURPDataset(Dataset):
    def __init__(self, df, word2idx, slot2id, intent2id, max_len=50):
        self.df = df
        self.word2idx = word2idx
        self.slot2id = slot2id
        self.intent2id = intent2id
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        tokens = self.df.iloc[idx]['text'].split()
        slots = self.df.iloc[idx]['slots'].split()
        intent = self.df.iloc[idx]['intent']

        length = len(tokens)
        input_ids = [self.word2idx.get(t, self.word2idx["<unk>"]) for t in tokens]
        slot_ids = [self.slot2id.get(s, 0) for s in slots]
        intent_id = self.intent2id[intent]

        pad_len = self.max_len - length
        input_ids += [self.word2idx["<pad>"]] * pad_len
        slot_ids += [0] * pad_len

        return torch.tensor(input_ids), torch.tensor(length), torch.tensor(slot_ids), torch.tensor(intent_id)

def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    lengths = torch.stack([item[1] for item in batch])
    slot_labels = torch.stack([item[2] for item in batch])
    intent_labels = torch.stack([item[3] for item in batch])
    return input_ids, lengths, slot_labels, intent_labels

batch_size = 32
train_dataset = SLURPDataset(train_df, word2idx, slot2id, intent2id)
val_dataset = SLURPDataset(val_df, word2idx, slot2id, intent2id)
test_dataset = SLURPDataset(test_df, word2idx, slot2id, intent2id)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


class JointRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, slot_label_size, intent_label_size, dropout=0.3):
        super(JointRNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.slot_classifier = nn.Linear(hidden_dim*2, slot_label_size)
        self.intent_classifier = nn.Linear(hidden_dim*2, intent_label_size)

        self._init_weights()

    def _init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

    def forward(self, input_ids, lengths):
        embeddings = self.embedding(input_ids)
        packed = nn.utils.rnn.pack_padded_sequence(embeddings, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, (hidden, cell) = self.encoder(packed)
        sequence_output, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True, total_length=input_ids.size(1))
        sequence_output = self.dropout(sequence_output)
        slot_logits = self.slot_classifier(sequence_output)
        hidden_cat = torch.cat((hidden[-2], hidden[-1]), dim=1)
        intent_logits = self.intent_classifier(hidden_cat)
        return slot_logits, intent_logits


vocab_size = max(word2idx.values()) + 1
embedding_dim = 128
hidden_dim = 256
slot_label_size = len(slot2id)
intent_label_size = len(intent2id)

model = JointRNNModel(vocab_size, embedding_dim, hidden_dim, slot_label_size, intent_label_size, dropout=0.5).to(device)
slot_loss_fn = nn.CrossEntropyLoss(ignore_index=0)
intent_loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)  # Added weight decay for regularization

def evaluate(model, loader):
    model.eval()
    all_intent_preds, all_intent_labels = [], []
    all_slot_preds, all_slot_labels = [], []

    with torch.no_grad():
        for input_ids, lengths, slot_labels, intent_labels in loader:
            input_ids, lengths = input_ids.to(device), lengths.to(device)
            slot_labels, intent_labels = slot_labels.to(device), intent_labels.to(device)

            slot_logits, intent_logits = model(input_ids, lengths)
            intent_preds = torch.argmax(intent_logits, dim=1)
            all_intent_preds.extend(intent_preds.cpu().tolist())
            all_intent_labels.extend(intent_labels.cpu().tolist())

            slot_preds = torch.argmax(slot_logits, dim=2)
            for i, l in enumerate(lengths):
                all_slot_preds.extend(slot_preds[i][:l].cpu().tolist())
                all_slot_labels.extend(slot_labels[i][:l].cpu().tolist())

    intent_acc = accuracy_score(all_intent_labels, all_intent_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_slot_labels, all_slot_preds, average='micro')
    print(f"Intent Acc: {intent_acc:.4f} | Slot P: {precision:.4f} R: {recall:.4f} F1: {f1:.4f}")
    model.train()
    return intent_acc, precision, recall, f1


def train(model, train_loader, val_loader, optimizer, epochs=10, lambda_intent=0.5):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, lengths, slot_labels, intent_labels in train_loader:
            input_ids, lengths = input_ids.to(device), lengths.to(device)
            slot_labels, intent_labels = slot_labels.to(device), intent_labels.to(device)

            optimizer.zero_grad()
            slot_logits, intent_logits = model(input_ids, lengths)

            slot_loss = slot_loss_fn(slot_logits.view(-1, slot_logits.shape[-1]), slot_labels.view(-1))
            intent_loss = intent_loss_fn(intent_logits, intent_labels)
            loss = slot_loss + lambda_intent * intent_loss

            if torch.isnan(loss) or torch.isinf(loss):
                print(f"Warning: NaN or Inf loss detected at epoch {epoch+1}, skipping batch")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.4f}")
        print("=== Validation ===")
        evaluate(model, val_loader)


train(model, train_loader, val_loader, optimizer, epochs=8, lambda_intent=1.0)

print("=== Test Set Evaluation ===")
evaluate(model, test_loader)


Epoch 1 | Train Loss: 6.0000
=== Validation ===
Intent Acc: 0.3002 | Slot P: 0.0506 R: 0.0506 F1: 0.0506
Epoch 2 | Train Loss: 4.1937
=== Validation ===
Intent Acc: 0.5863 | Slot P: 0.0859 R: 0.0859 F1: 0.0859
Epoch 3 | Train Loss: 2.8905
=== Validation ===
Intent Acc: 0.7293 | Slot P: 0.1115 R: 0.1115 F1: 0.1115
Epoch 4 | Train Loss: 2.0693
=== Validation ===
Intent Acc: 0.7797 | Slot P: 0.1293 R: 0.1293 F1: 0.1293
Epoch 5 | Train Loss: 1.5759
=== Validation ===
Intent Acc: 0.7884 | Slot P: 0.1419 R: 0.1419 F1: 0.1419
Epoch 6 | Train Loss: 1.2451
=== Validation ===
Intent Acc: 0.8086 | Slot P: 0.1517 R: 0.1517 F1: 0.1517
Epoch 7 | Train Loss: 1.0401
=== Validation ===
Intent Acc: 0.7992 | Slot P: 0.1551 R: 0.1551 F1: 0.1551
Epoch 8 | Train Loss: 0.8555
=== Validation ===
Intent Acc: 0.8120 | Slot P: 0.1597 R: 0.1597 F1: 0.1597
=== Test Set Evaluation ===
Intent Acc: 0.8031 | Slot P: 0.1562 R: 0.1562 F1: 0.1562


(0.803065490013934,
 0.15618755827893965,
 0.15618755827893965,
 0.15618755827893965)